In [8]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import pudl, ferc1, eia923, settings, constants
from pudl import models, models_ferc1, models_eia923
from pudl import clean_eia923, clean_ferc1, clean_pudl
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
pudl_engine  = pudl.connect_db()

In [12]:
#Pull in the generation table
g_tbl = models.PUDLBase.metadata.tables['generation_eia923']
g_select = sa.sql.select([g_tbl,]) #creates a sql Select object
g = pd.read_sql(g_select, pudl_engine) #converts sql object to pandas dataframe
#Set the datetimeindex
g = g.set_index(pd.DatetimeIndex(g['report_date'])) #sets report_date as index

In [14]:
#groupby plant_id and by year
g_yr = g.groupby([pd.TimeGrouper(freq='A'), 'plant_id', 'generator_id'])
#sum net_gen by year by plant
g_net_gen_gen = pd.DataFrame(g_yr.net_generation_mwh.sum())
g_net_gen_gen = g_net_gen_gen.reset_index(level=['generator_id'])
g_net_gen_gen

generator_id  net_generation_mwh
report_date plant_id                                 
2009-12-31  3                   1           221908.00
            3                   2           394031.00
            3                   3          1286393.00
            3                   4          1626547.00
            3                   5          4513101.00
            3                A1ST          1122697.00
            3                A2ST          1033733.00
            7                   1           212068.00
            7                   2            51262.00
            8                  10          3933248.00
            8                   6            98792.00
            8                   7            89753.00
            8                   8           566465.00
            8                   9           519196.00
            10                  1          1543167.00
            10                  2           821517.00
            26                  1           856407.00
            26                  2          1022033.00
            26                  3           657461.00
            26                  5          4706644.00
            26                ST4           790519.00
            47                  1           634534.00
            47                  2           424711.00
            47                  3           271306.00
            47                  4           454250.00
            47                  5          1154230.00
            50                  1            96134.00
            50                  2           111334.00
            50                  3            40763.00
            50                  4           162681.00
...                           ...                 ...
2016-12-31  58260            CT03                0.00
            58260            ST01          4206354.00
            58420            GEN1          1344962.00
            58420            GEN2           778894.00
            58426            GEN1          1158570.92
            58426            GEN2          1200521.04
            58503            GEN1           653485.00
            58503            GEN2           408282.00
            58557             GT1           521721.00
            58557             GT2           572682.00
            58557             ST1           864861.00
            58560            STG1           199787.00
            58697             CA1          1644040.00
            58697             CT1          1310111.00
            58697             CT2          1346797.00
            58944              G1            18752.00
            59254           GENS1            52536.17
            59254           GENT1            32347.56
            59254           GENT2            33741.45
            59254           GENT3            33873.37
            59254           GENT4            34612.51
            59338            CTG1           352863.00
            59338            CTG2           168324.00
            59338            CTG3           218810.00
            59338            CTG4           281232.00
            59338            ST10           339927.00
            59338             ST9           319657.00
            59784            CTG5           602776.00
            59784            CTG6           738478.00
            59784            ST11           892986.00

[27030 rows x 2 columns]

In [15]:
#groupby plant_id and by year
g_net_gen_plant = g.groupby([pd.TimeGrouper(freq='A'), 'plant_id'])
#sum net_gen by year by plant and convert to datafram
g_net_gen_plant =  pd.DataFrame(g_net_gen_plant.net_generation_mwh.sum())
g_net_gen_plant

net_generation_mwh
report_date plant_id                    
2009-12-31  3               1.019841e+07
            7               2.633300e+05
            8               5.207454e+06
            10              2.364684e+06
            26              8.033064e+06
            47              2.939031e+06
            50              4.688202e+06
            51              4.465132e+06
            56              3.026254e+06
            59              5.916670e+05
            60              4.664970e+05
            79              2.093790e+05
            87              1.835862e+06
            108             2.655821e+06
            113             7.236124e+06
            116             7.872800e+04
            117             3.445720e+05
            118             6.610200e+04
            120             3.187580e+05
            126             6.822990e+05
            127             2.813612e+06
            130             1.461985e+07
            136             7.097411e+06
            141             6.012300e+04
            147             2.414980e+05
            160             2.041529e+06
            165             6.840435e+06
            167             2.010700e+04
            168             2.263000e+03
            169             4.452600e+04
...                                  ...
2016-12-31  57664           2.552517e+06
            57759           5.541734e+04
            57839           4.263133e+06
            57842           5.409940e+05
            57898           4.341340e+05
            57901           1.094088e+06
            57909           3.904510e+05
            57919           1.387590e+05
            57978           5.026250e+05
            58001           4.170955e+06
            58005           2.960254e+06
            58007           2.277812e+05
            58023           1.014350e+05
            58054           4.780460e+05
            58079           3.974936e+06
            58108           3.651440e+05
            58109           4.269686e+04
            58124           8.702400e+04
            58215           5.806522e+06
            58260           5.177729e+06
            58420           2.123856e+06
            58426           2.359092e+06
            58503           1.061767e+06
            58557           1.959264e+06
            58560           1.997870e+05
            58697           4.300948e+06
            58944           1.875200e+04
            59254           1.871111e+05
            59338           1.680813e+06
            59784           2.234240e+06

[11459 rows x 1 columns]

In [16]:
#Merge the summed net generation by generator with the summed net generation by plant
g_merged = g_net_gen_gen.merge(g_net_gen_plant, how="left", left_index=True, right_index=True)
g_merged['proportion_of_generation'] = (g_merged.net_generation_mwh_x/g_merged.net_generation_mwh_y)
#Remove the net generation columns
g_merged = g_merged.drop(['net_generation_mwh_x','net_generation_mwh_y'], axis=1)
g_merged

generator_id  proportion_of_generation
report_date plant_id                                       
2009-12-31  3                   1                  0.021759
            3                   2                  0.038637
            3                   3                  0.126137
            3                   4                  0.159490
            3                   5                  0.442530
            3                A1ST                  0.110085
            3                A2ST                  0.101362
            7                   1                  0.805332
            7                   2                  0.194668
            8                  10                  0.755311
            8                   6                  0.018971
            8                   7                  0.017235
            8                   8                  0.108780
            8                   9                  0.099702
            10                  1                  0.652589
            10                  2                  0.347411
            26                  1                  0.106610
            26                  2                  0.127228
            26                  3                  0.081844
            26                  5                  0.585909
            26                ST4                  0.098408
            47                  1                  0.215899
            47                  2                  0.144507
            47                  3                  0.092311
            47                  4                  0.154558
            47                  5                  0.392725
            50                  1                  0.020506
            50                  2                  0.023748
            50                  3                  0.008695
            50                  4                  0.034700
...                           ...                       ...
2016-12-31  58260            CT03                  0.000000
            58260            ST01                  0.812394
            58420            GEN1                  0.633264
            58420            GEN2                  0.366736
            58426            GEN1                  0.491109
            58426            GEN2                  0.508891
            58503            GEN1                  0.615469
            58503            GEN2                  0.384531
            58557             GT1                  0.266284
            58557             GT2                  0.292294
            58557             ST1                  0.441421
            58560            STG1                  1.000000
            58697             CA1                  0.382251
            58697             CT1                  0.304610
            58697             CT2                  0.313140
            58944              G1                  1.000000
            59254           GENS1                  0.280775
            59254           GENT1                  0.172879
            59254           GENT2                  0.180328
            59254           GENT3                  0.181033
            59254           GENT4                  0.184984
            59338            CTG1                  0.209936
            59338            CTG2                  0.100144
            59338            CTG3                  0.130181
            59338            CTG4                  0.167319
            59338            ST10                  0.202240
            59338             ST9                  0.190180
            59784            CTG5                  0.269790
            59784            CTG6                  0.330528
            59784            ST11                  0.399682

[27030 rows x 2 columns]

In [ ]:
#Pull in the neg_generation data in MMBtu from generation_fuel table

In [18]:
#Pull in the fuel_receipts_cost table
gf_tbl = models.PUDLBase.metadata.tables['generation_fuel_eia923']
gf_select = sa.sql.select([gf_tbl,])
gf = pd.read_sql(gf_select, pudl_engine)
#Set the datetimeindex
gf = gf.set_index(pd.DatetimeIndex(gf['report_date']))
#groupby plant_id and by year
gf_yr_fuel_mmbtu = gf.groupby([pd.TimeGrouper(freq='A'), 'plant_id'])
#sum fuel cost by year by plant
gf_yr_fuel_mmbtu = gf_yr_fuel_mmbtu.fuel_consumed_total_mmbtu.sum()
#Convert back into a dataframe
gf_yr_fuel_mmbtu_df = pd.DataFrame(gf_yr_fuel_mmbtu)
gf_yr_fuel_mmbtu_df

fuel_consumed_total_mmbtu
report_date plant_id                           
2009-12-31  2                         2758750.0
            3                       122482205.0
            4                         8427915.0
            7                         3886265.0
            8                        52697640.0
            9                          225529.0
            10                       25383733.0
            11                        2609220.0
            12                        2113616.0
            13                        4169112.0
            14                        4996231.0
            15                        7466001.0
            16                        4825247.0
            17                        6372224.0
            18                        4316622.0
            19                        2977181.0
            20                        2830019.0
            21                        1785758.0
            26                       79700771.0
            30                            165.0
            34                         651196.0
            38                        3289482.0
            46                      269988673.0
            47                       32607544.0
            48                        7532350.0
            49                       15177083.0
            50                       51648015.0
            51                       51926892.0
            53                          96323.0
            54                        2883466.0
...                                         ...
2016-12-31  60058                      227514.0
            60059                     3482435.0
            60064                      568757.0
            60069                     6971024.0
            60082                      544891.0
            60087                     4474956.0
            60092                      785479.0
            60093                     1853428.0
            60104                     8008627.0
            60127                       86807.0
            60143                      133541.0
            60233                      290371.0
            60259                      411276.0
            60262                     2763707.0
            60285                      316985.0
            60314                      870105.0
            60326                     1040979.0
            60338                      431237.0
            60339                      931630.0
            60342                      904614.0
            60351                       81970.0
            60355                      288283.0
            60389                      410652.0
            60405                      154341.0
            60406                      450500.0
            60413                       54423.0
            60419                     1465003.0
            60445                      217645.0
            60449                       47900.0
            60486                       87775.0

[47295 rows x 1 columns]

In [24]:
merged = g_merged.merge(gf_yr_fuel_mmbtu_df, how="left", left_index=True, right_index=True)
merged['fuel_consumed_mmbtu'] = (merged.fuel_consumed_total_mmbtu*merged.proportion_of_generation)
merged = merged.drop(['fuel_consumed_total_mmbtu'], axis=1)


In [26]:
merged

generator_id  proportion_of_generation  \
report_date plant_id                                          
2009-12-31  3                   1                  0.021759   
            3                   2                  0.038637   
            3                   3                  0.126137   
            3                   4                  0.159490   
            3                   5                  0.442530   
            3                A1ST                  0.110085   
            3                A2ST                  0.101362   
            7                   1                  0.805332   
            7                   2                  0.194668   
            8                  10                  0.755311   
            8                   6                  0.018971   
            8                   7                  0.017235   
            8                   8                  0.108780   
            8                   9                  0.099702   
            10                  1                  0.652589   
            10                  2                  0.347411   
            26                  1                  0.106610   
            26                  2                  0.127228   
            26                  3                  0.081844   
            26                  5                  0.585909   
            26                ST4                  0.098408   
            47                  1                  0.215899   
            47                  2                  0.144507   
            47                  3                  0.092311   
            47                  4                  0.154558   
            47                  5                  0.392725   
            50                  1                  0.020506   
            50                  2                  0.023748   
            50                  3                  0.008695   
            50                  4                  0.034700   
...                           ...                       ...   
2016-12-31  58260            CT03                  0.000000   
            58260            ST01                  0.812394   
            58420            GEN1                  0.633264   
            58420            GEN2                  0.366736   
            58426            GEN1                  0.491109   
            58426            GEN2                  0.508891   
            58503            GEN1                  0.615469   
            58503            GEN2                  0.384531   
            58557             GT1                  0.266284   
            58557             GT2                  0.292294   
            58557             ST1                  0.441421   
            58560            STG1                  1.000000   
            58697             CA1                  0.382251   
            58697             CT1                  0.304610   
            58697             CT2                  0.313140   
            58944              G1                  1.000000   
            59254           GENS1                  0.280775   
            59254           GENT1                  0.172879   
            59254           GENT2                  0.180328   
            59254           GENT3                  0.181033   
            59254           GENT4                  0.184984   
            59338            CTG1                  0.209936   
            59338            CTG2                  0.100144   
            59338            CTG3                  0.130181   
            59338            CTG4                  0.167319   
            59338            ST10                  0.202240   
            59338             ST9                  0.190180   
            59784            CTG5                  0.269790   
            59784            CTG6                  0.330528   
            59784            ST11                  0.399682   

                      fuel_consumed_mmbtu  
report_